# Langchain Hybrid Search
`Hybrid Search:` Combining Traditional **Keyword-Based Search** with Modern **Vector Search**

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

pc_api_key = os.getenv("PINECONE_API_KEY")

### Creating a Pinecone Index (we can also create manually from Pinecone website)

In [4]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
from pinecone import Pinecone, ServerlessSpec

# Index Name
index_name = "langchain-pinecone-hybrid-search"
#Initializing pinecone client
pc = Pinecone(api_key=pc_api_key)
# Create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
                    dimension=384,  # Dimension of dense vector (we'll use HF Embeddings)
                    metric="dotproduct",  # sparse values are only supported for dotproduct
                    spec=ServerlessSpec(cloud="aws", region="us-east-1")
                    )

In [5]:
index = pc.Index(index_name)
index

### 1. Huggingface Vector Embeddings

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### 2. Sparse BM25Encoder

In [13]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [14]:
sentences = [
    "In 2022, I visited Lahore",
    "In 2024, I visited Islamabad",
    "In 2020, I visited Multan",
]

# Applying TFIDF on sentences
bm25_encoder.fit(sentences)

# Storing values to a JSON file
bm25_encoder.dump("bm25_values.json")

# Load the BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 48.98it/s]


### Adding Text Sentences in Pinecone Index

In [15]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    index=index,
    sparse_encoder=bm25_encoder
)

In [16]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:06<00:00,  6.28s/it]


In [17]:
retriever.invoke("Which city did I visit last?")

[Document(page_content='In 2022, I visited Lahore'),
 Document(page_content='In 2020, I visited Multan'),
 Document(page_content='In 2024, I visited Islamabad')]